In [1]:
# Load the KAG conversion data and do a quick exploratory summary to answer:
# Which age and gender groups drive highest sales (Approved_Conversion), highest ad spend (Spent), and their conversion efficiency (Approved_Conversion per Click and per Spent)?
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Read data
file_path = 'KAG_conversion_data_raw.csv'
df_conv = pd.read_csv(file_path, encoding='ascii')
print('Loaded KAG_conversion_data_raw.csv with shape:', df_conv.shape)
print(df_conv.head())

In [ ]:
# Basic cleaning: ensure numeric types
num_cols = ['Impressions', 'Clicks', 'Spent', 'Total_Conversion', 'Approved_Conversion']
for c in num_cols:
    df_conv[c] = pd.to_numeric(df_conv[c], errors='coerce')

In [ ]:
# Drop rows with missing critical fields
df_clean = df_conv.dropna(subset=['age', 'gender', 'Spent', 'Approved_Conversion', 'Clicks'])
print('After cleaning, shape:', df_clean.shape)

# Aggregate by age and gender
agg = df_clean.groupby(['age', 'gender'], as_index=False).agg({
    'Spent': 'sum',
    'Clicks': 'sum',
    'Approved_Conversion': 'sum'
})



In [ ]:
# Compute efficiency metrics
agg['Conv_per_Click'] = np.where(agg['Clicks'] > 0, agg['Approved_Conversion'] / agg['Clicks'], np.nan)
agg['Conv_per_Spent'] = np.where(agg['Spent'] > 0, agg['Approved_Conversion'] / agg['Spent'], np.nan)

# Rank groups
agg['Rank_Sales'] = agg['Approved_Conversion'].rank(ascending=False, method='dense')
agg['Rank_Spend'] = agg['Spent'].rank(ascending=False, method='dense')



In [ ]:
# Identify top groups
top_sales = agg.sort_values('Approved_Conversion', ascending=False).head(5)
top_spend = agg.sort_values('Spent', ascending=False).head(5)
print('Top groups by sales (Approved_Conversion):')
print(top_sales[['age','gender','Approved_Conversion','Spent','Clicks','Conv_per_Click','Conv_per_Spent']])
print('Top groups by spend (Spent):')
print(top_spend[['age','gender','Approved_Conversion','Spent','Clicks','Conv_per_Click','Conv_per_Spent']])



In [ ]:
# Visualizations
plt.figure(figsize=(9,5))
sns.barplot(data=agg.sort_values('Approved_Conversion', ascending=False), x='age', y='Approved_Conversion', hue='gender')
plt.title('Approved Conversions by Age and Gender')
plt.ylabel('Approved Conversions')
plt.xlabel('Age Group')
plt.legend(title='Gender')
plt.tight_layout()
plt.show()

plt.figure(figsize=(9,5))
sns.barplot(data=agg.sort_values('Spent', ascending=False), x='age', y='Spent', hue='gender')
plt.title('Ad Spend by Age and Gender')
plt.ylabel('Total Spend')
plt.xlabel('Age Group')
plt.legend(title='Gender')
plt.tight_layout()
plt.show()

plt.figure(figsize=(9,5))
sns.barplot(data=agg.sort_values('Conv_per_Spent', ascending=False), x='age', y='Conv_per_Spent', hue='gender')
plt.title('Conversion Efficiency (Approved per Dollar) by Age and Gender')
plt.ylabel('Approved per Dollar')
plt.xlabel('Age Group')
plt.legend(title='Gender')
plt.tight_layout()
plt.show()

plt.figure(figsize=(9,5))
sns.barplot(data=agg.sort_values('Conv_per_Click', ascending=False), x='age', y='Conv_per_Click', hue='gender')
plt.title('Conversion Rate (Approved per Click) by Age and Gender')
plt.ylabel('Approved per Click')
plt.xlabel('Age Group')
plt.legend(title='Gender')
plt.tight_layout()
plt.show()



In [ ]:
# Provide a concise table sorted by sales and spend for reference
summary = agg.sort_values(['Approved_Conversion','Spent'], ascending=[False, False]).reset_index(drop=True)
print('Summary by age x gender (top 10 by sales):')
print(summary.head(10))